# DWT (1D): DT ANALYSIS FILTER $\rightarrow \downarrow 2$
- **Keras**

[=] $IN_{rank3tensor}\rightarrow$ DWT ANALYSIS FILTER $\rightarrow \downarrow2$  $\rightarrow OUT_{rank3tensor}$ 

[=]  Forward Transform Layer for a rank 3 tensor

[+]  Rank 3 INPUT TENSOR $B\times N\times 1$ (batch_size:b, input_len=N, channels:1)

[=]  $h_{0}[n]$, $h_{1}[n]$ realization for tensor procssing

[+]  1D CONVOLUTION: USING 1D DWT ANALYSIS FILTER impulse responses 

[+]  OTHOGONAL WAVELET FAMILES (db/sym/coif)

[=]  For DWT of vectors (1D)

[?]  $B\times N\times ?channel$  -do-            (batch_size:$B$, input_len=$N$, channels:$?multiple!!$)

<br />
    
--- 
Kishore Kr. Tarafdar, --@k 10/07/2022

<br />
<br />

COMPATIBLE WAVELETS:

<br />
<br />
<br />
<br />

---

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
from keras.layers import Lambda
import numpy as np
import import_ipynb

2022-09-26 03:33:53.879162: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-26 03:33:53.879204: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
## List of Lists to a List 
LOL2L = lambda LOL:np.array([_ for sublist in LOL for _ in sublist])

## Relative error: Assign True as 1 and False as 0 in a list, sum up and divide by length of list
err = lambda p: np.sum([1 if _==False else 0 for _ in list(p)])/len(p) 

<br />
<br />

## 1D Conv. layer
- DT filter layer
$$\rightarrow h[n] \rightarrow$$ 

In [3]:
class DWTConv1DLayer(layers.Conv1D):
    """DWT (1D) conv. layer, filtering only --@k"""
    def convolution_op(self, inputs, kernel):
        return tf.nn.conv1d(inputs,
            kernel,
            padding="VALID",
            stride=list(self.strides),
            name=self.__class__.__name__,
        )

## 2BPRFB Analysis branch (DWT)
- Filter and downsample by 2 layer

$$\rightarrow h[n] \rightarrow \downarrow 2 \rightarrow $$

In [4]:
# Filter and downsample by 2 ---all in 1
class AnalysisBranch1D(keras.layers.Layer):
    """DWT analysis branch: DT Filter and downsample by 2 --@k"""
    def __init__(self, hn:list): 
        super(AnalysisBranch1D, self).__init__()
        
        filt_len = len(hn)
        paddings = tf.constant([[0,0], [filt_len - 1, filt_len - 1],[0,0]])
        
        #filter: kernel initializer
        self.h = tf.keras.initializers.Constant(hn)
        
        #layers
        self.pad = Lambda(lambda x: tf.pad(x, paddings, "SYMMETRIC"))
        self.dwt_filter = DWTConv1DLayer(1, kernel_size=filt_len, kernel_initializer=self.h, trainable=False) 
        self.downsample2 = Lambda(lambda x: x[:,1::2])

    def call(self, inputs):
        x = self.pad(inputs)
        x = self.dwt_filter(x)
        x = self.downsample2(x)
        return x#self.downsample(x)